In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from datetime import datetime
from collections import defaultdict

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
# joint_df = pd.read_csv('./joint_clean_data/joint.csv', index_col=0)

In [6]:
# goal_domain = joint_df[['goal_domain']]

In [7]:
# goal_domain.to_csv('goal_domains.csv')

In [8]:
goal_domain = pd.read_csv('goal_domains.csv', index_col=0)

In [9]:
goal_domain.dropna(inplace=True)

In [10]:
goal_domain.head()

,goal_domain
0,Творчество и создание нового (Изобразительное ...
1,"Универсальные компетенции, гибкие навыки (soft..."
2,"Иностранные языки, гуманитарные знания и навык..."
3,"Прикладные знания и навыки, ручной труд (инфор..."
4,"Универсальные компетенции, гибкие навыки (soft..."


In [11]:
cv = CountVectorizer(max_df=0.5)
dtm = cv.fit_transform(goal_domain['goal_domain'])

In [12]:
LDA = LatentDirichletAllocation(n_components=11,
                               batch_size=256,
                               learning_decay=0.2) #playaround # 11 because we have 11 labels

In [13]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=256, learning_decay=0.2, n_components=11)

In [14]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]]) #adjust
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['продажи', 'маркетинг', 'деньги', 'стартапы', 'проекты', 'рабочих', 'менеджмент', 'процессов', 'организация', 'нормативное', 'регулирование', 'работа', 'финансы', 'бизнес', 'личные']


THE TOP 15 WORDS FOR TOPIC #1
['профилактика', 'здоровья', 'улучшение', 'внешний', 'вид', 'правильное', 'веса', 'питание', 'контроль', 'привычки', 'полезные', 'самочувствие', 'спорт', 'внешность', 'зож']


THE TOP 15 WORDS FOR TOPIC #2
['общения', 'командной', 'работы', 'самоорганизация', 'тайм', 'планирование', 'менеджмент', 'осознанность', 'мышления', 'развитие', 'гибкие', 'универсальные', 'soft', 'skills', 'компетенции']


THE TOP 15 WORDS FOR TOPIC #3
['садоводство', 'хозяйству', 'дому', 'транспортного', 'средства', 'вождение', 'работа', 'по', 'строительство', 'ручного', 'труда', 'ручной', 'труд', 'прикладные', 'знания']


THE TOP 15 WORDS FOR TOPIC #4
['экономика', 'информационные', 'технологии', 'жизни', 'новых', 'деньги', 'нормативное', 'регулирование', 'бизнес', 'ра

In [540]:
topics_tesaurus = {
    'label_attractor_knowledge':['знан','зако','прав', 'нау', 'образов', 'биолог','философ','социолог','изуч','литерат','язык','учеб','экономика','экология','кулинар','энергети','электро','обуч','безопасн','хими','стоматол','педиат','вет','логопед','строител','ремонт','культур','учёб','гомеопат'],
    'label_attractor_hard_skill':['математи', 'программирова','ии','ит','it','дизайн','графи','создан','юрис','медицин', 'психо','педагоги', 'предпринимател','воспита','разработ', 'по', 'прикладн','инженер','hard','информац','иннов', 'ритор','автомех','флори','фарма','косметол','логист','цифр','машин','металл','механ','гео','экос','агро','меха','дефект','свар','хлебопеч','шахмат','освоен'], 
    'label_attractor_soft_skill':['soft', 'саморазвит', 'дисциплин', 'общен','дипломат','коммуни','soft','commun','врем','мышл'],
    'label_attractor_tool':['техн','1с','cdo','edtech','corporate finullce','нти','оборудован'],
    'label_attractor_community':['обществ', 'семья','семьей','люд', 'семью', 'семейн','worldskill', 'wsr', 'социал','стран','домашн','волонт','отношен','ислам','религ','рсо','предложени','фестивал'],
    'label_attractor_subjectivity':[ 'благ','личност','личн','мисси', 'эмоц','развле','собствен','независ','ден','💰','само','заработ','успех','себя','материаль'],
    'label_attractor_habits':['привыч'],
    'label_attractor_career':['карьер', 'работа','сфер','отрасл','бизнес','государст','финанс','жкх','управл', 'маркетинг','руковод', 'здравоохран', 'дело', 'проф', 'менедж','врач','деятел','производ','админ','парик','услуг','сервис','работ','журналис','организац','хозяйс','hr','консал','продвижен','нтр','труд','экспанс'],
    'label_attractor_fixing':['вуз','колледж','гму'],
    'label_attractor_art':['худож', 'искусств', 'фото', 'творч','мечт','жизн', 'духов','хобби','мир','просвещ','путешеств','эстет','решени'],
    'label_attractor_health':['здоров', 'спорт', 'зож', 'тури','питан','внешн','вес','полезн','крас'],
    'label_attractor_undefined':['нет','все','всё', 'не','секрет','ок','-','*','1','угодн','никак', '.','любой','любая','чромомл','общее','тема']

}

In [541]:
def add_cluster(x):
    group = np.nan
    stop = 0
    for key, values in topics_tesaurus.items():
        for val in values:
            if val in x:
                group = key
                stop = 1
                break
        if stop == 1:
            break
    if stop == 0:
        return group
    return group

In [542]:
goal_domain['goal_domain'] = goal_domain['goal_domain'].apply(lambda x: x.lower())

In [543]:
goal_domain['topic'] = goal_domain['goal_domain'].apply(lambda x: add_cluster(x))

In [544]:
goal_domain.tail(20) #playaround head-tail, number

,goal_domain,topic
17998,"спорт, зож, внешность и самочувствие (зож, пол...",label_attractor_knowledge
17999,"личные финансы, работа, бизнес и нормативное р...",label_attractor_knowledge
18000,математика и it (программирование и освоение к...,label_attractor_hard_skill
18001,предпринимательство,label_attractor_hard_skill
18002,педагогика,label_attractor_hard_skill
18003,"универсальные компетенции, гибкие навыки (soft...",label_attractor_knowledge
18004,"иностранные языки, гуманитарные знания и навык...",label_attractor_knowledge
18005,математика и it (программирование и освоение к...,label_attractor_hard_skill
18006,творчество и создание нового (проект),label_attractor_hard_skill
18007,воспитательная работа,label_attractor_hard_skill


In [545]:
goal_domain[goal_domain['topic'].isna() == True].shape

(9, 2)

In [546]:
goal_domain[goal_domain['topic'].isna() == True].head(50)

,goal_domain,topic
3041,интелект,NaN
3678,внутренние качества человека,NaN
4684,инжиниринг,NaN
5960,начальные классы,NaN
8456,эксплуатация беспилотных авиационных систем,NaN
9804,стамотология,NaN
11493,а,NaN
15022,мндицина,NaN
17608,к своему будущему,NaN


In [547]:
import pickle

In [548]:
with open('topics_tesaurus.pickle', 'wb') as f:
    pickle.dump(topics_tesaurus, f)